In [1]:
%matplotlib inline
import dgl
import glob
import pprint
import numpy as np
import awkward as ak
import networkx as nx
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from os import path
from tqdm import tqdm
from pathlib import Path
from trainresults import TrainResults
from train_eval_func import train, evaluate
from tensorflow import keras
from copy import deepcopy
from dgl.data import DGLDataset
from dgl.dataloading import GraphDataLoader
from TauGraphDatasetInfo import TauGraphDatasetInfo
from TauGraphDataset import TauGraphDataset, GetNodeFeatureVectors, GetEdgeFeatureVectors
from TauGraphDataset import GetNeighborNodes, GetEdgeList, GetEdgeFeatureVectorsFromSourceNode, Graph2FlatZeropaddedList

plt.rcParams.update({'font.size': 20})
plt.rcParams['text.usetex'] = True
lw = 2
xyLabelFontSize = 20
xLabelPad = 10
yLabelPad = 15
pp = pprint.PrettyPrinter()

Using backend: pytorch


In [2]:
def getDatasetNames(datasetDir):
    files = glob.glob(datasetDir + '/*.json', recursive=True)
    files.sort()
    datasetDirectories = [path.dirname(file) for file in files]
    datasetnames = [path.normpath(dir).split(path.sep)[-1] for dir in datasetDirectories]
    return datasetDirectories, datasetnames

In [3]:
datasetDir = '/ceph/aissac/ntuple_for_graphs/prod_2018_v2_processed_v5_THESIS/trimmed_200000_and_cut_puppiWeightNoLep_greater_0_and_deltaR_smaller_0point5/Graphs_DYJetsToLL_M-50_genuineTaus_and_jets'
datasetDirs, datasetNames = getDatasetNames(datasetDir)
print(datasetDirs)
print(datasetNames)

['/ceph/aissac/ntuple_for_graphs/prod_2018_v2_processed_v5_THESIS/trimmed_200000_and_cut_puppiWeightNoLep_greater_0_and_deltaR_smaller_0point5/Graphs_DYJetsToLL_M-50_genuineTaus_and_jets']
['Graphs_DYJetsToLL_M-50_genuineTaus_and_jets']


In [4]:
dataset = TauGraphDataset(datasetNames[0], datasetDirs[0])
print(dataset)

Done loading data from cached files.


In [5]:
print(f'name: {datasetNames[0]},\n directory: {datasetDirs[0]}')
graph, label = dataset[0]
print(graph)
print(f'label: {label}')
print(f'graph classes: {dataset.graphClasses}')
print(f'dataset graph count: {dataset.num_graphs}')
print(f'nodeFeatKeys: {dataset.nodeFeatKeys}')
print(f'edgeFeatKeys: {dataset.edgeFeatKeys}')
print(f'graphFeatkeys: {dataset.graphFeatKeys}')
print(f'max node count: {dataset.maxNodeCount}')
print(f'min node count: {dataset.minNodeCount}')

name: Graphs_DYJetsToLL_M-50_genuineTaus_and_jets,
 directory: /ceph/aissac/ntuple_for_graphs/prod_2018_v2_processed_v5_THESIS/trimmed_200000_and_cut_puppiWeightNoLep_greater_0_and_deltaR_smaller_0point5/Graphs_DYJetsToLL_M-50_genuineTaus_and_jets
Graph(num_nodes=25, num_edges=600,
      ndata_schemes={'feat': Scheme(shape=(7,), dtype=torch.float64)}
      edata_schemes={'feat': Scheme(shape=(3,), dtype=torch.float32)})
label: 1
graph classes: ['0', '1']
dataset graph count: 200000
nodeFeatKeys: ['pt', 'eta', 'phi', 'mass', 'charge', 'particleType', 'summand']
edgeFeatKeys: ['deltaEta', 'deltaPhi', 'deltaR']
graphFeatkeys: ['nodeCount', 'tau_byDeepTau2017v2p1VSjet']
max node count: 81
min node count: 2


In [6]:
graphs, labels = dataset[:]
g = graphs[0]
nFeatDim = dataset.dim_nfeats
eFeatDim = dataset.dim_efeats
maxNodeCount = dataset.maxNodeCount
print(g)
print(f'nFeatDim: {nFeatDim}')
print(f'eFeatDim: {eFeatDim}')
print(f'maxNodeCount: {maxNodeCount}')
print()

nodeAndEdgeFeaturePaddedDim = nFeatDim + eFeatDim * (maxNodeCount - 1)
print(f'node + edge features dim per Node (includes zero padding if nodecount<maxnodecount):\n',
      f'nFeatDim + eFeatDim * (maxNodeCount-1) = {nFeatDim} + {eFeatDim} * {maxNodeCount - 1} = ',
      f'{nodeAndEdgeFeaturePaddedDim}')
print(f'The (maxNodeCount-1) comes from fully connected graphs without self-loops')
print()
print(f'only node features dim per Node: nFeatDim={nFeatDim}')
print()

nodeAndEdgeFeaturePaddedDimInputSize = nodeAndEdgeFeaturePaddedDim * maxNodeCount
nodeFeaturePaddedDimInputSize = nFeatDim * maxNodeCount
print(f'node + edge features with zero padding to fill until maxNodeCount leads to inputsize: {nodeAndEdgeFeaturePaddedDimInputSize}')
useEdgeFeat = True
temp = np.array(Graph2FlatZeropaddedList(g, nFeatDim, eFeatDim, maxNodeCount, useEdgeFeat), dtype=np.float32)
print(f'check example: node + edge features list size: {len(temp)}')
print()
print(f'only node features with zero padding to fill until maxNodeCount leads to inputsize: {nodeFeaturePaddedDimInputSize}')
useEdgeFeat = False
temp = np.array(Graph2FlatZeropaddedList(g, nFeatDim, eFeatDim, maxNodeCount, useEdgeFeat), dtype=np.float32)
print(f'check example: only node features list size: {len(temp)}')

Graph(num_nodes=25, num_edges=600,
      ndata_schemes={'feat': Scheme(shape=(7,), dtype=torch.float64)}
      edata_schemes={'feat': Scheme(shape=(3,), dtype=torch.float32)})
nFeatDim: 7
eFeatDim: 3
maxNodeCount: 81

node + edge features dim per Node (includes zero padding if nodecount<maxnodecount):
 nFeatDim + eFeatDim * (maxNodeCount-1) = 7 + 3 * 80 =  247
The (maxNodeCount-1) comes from fully connected graphs without self-loops

only node features dim per Node: nFeatDim=7

node + edge features with zero padding to fill until maxNodeCount leads to inputsize: 20007
check example: node + edge features list size: 20007

only node features with zero padding to fill until maxNodeCount leads to inputsize: 567
check example: only node features list size: 567


In [7]:
def getInputData(dgldataset, useEdgeFeatures):
    inputs = [] 
    graphs, labels = dgldataset[:]
    maxNodeCount = dgldataset.maxNodeCount
    nFeatDim = dgldataset.dim_nfeats
    eFeatDim = dgldataset.dim_efeats
    
    import time
    start = time.time()
    it = 0
    
    for i in tqdm(range(len(graphs))):
        inputs.append(Graph2FlatZeropaddedList(graphs[i], nFeatDim, eFeatDim, maxNodeCount, useEdgeFeatures))

    # Stack all inputs_ vertically
    print(type(inputs))
    inputs = np.array(inputs, dtype=np.float32)
    print(inputs)
    print(type(inputs))
    inputs = np.vstack(inputs)
    print(inputs)
    

    # Stack all labels_ horizontally
    labels = np.hstack(labels)

    print("Input shape: ", inputs.shape)
    print("Labels shape: ", labels.shape)

    labels = tf.keras.utils.to_categorical(labels)
    print(labels.shape)
    print(labels[0])
    end = time.time() - start
    print(f'graphs to flattened zero padded list took {end:.2f} seconds ({end/60:.2f} minutes)')
    return inputs, labels

In [8]:
inputs, labels = getInputData(dataset, False)

100%|██████████| 200000/200000 [01:37<00:00, 2045.07it/s]


<class 'list'>
[[ 0.6791992   1.238197   -0.6319463  ...  0.          0.
   0.        ]
 [ 2.7011719  -0.5762505   2.2737963  ...  0.          0.
   0.        ]
 [ 4.2695312  -0.781518    0.42511293 ...  0.          0.
   0.        ]
 ...
 [ 0.66308594 -0.8608051   1.7206644  ...  0.          0.
   0.        ]
 [ 1.4853516   1.7232581  -3.10292    ...  0.          0.
   0.        ]
 [ 0.8852539   1.0530717  -2.2728148  ...  0.          0.
   0.        ]]
<class 'numpy.ndarray'>
[[ 0.6791992   1.238197   -0.6319463  ...  0.          0.
   0.        ]
 [ 2.7011719  -0.5762505   2.2737963  ...  0.          0.
   0.        ]
 [ 4.2695312  -0.781518    0.42511293 ...  0.          0.
   0.        ]
 ...
 [ 0.66308594 -0.8608051   1.7206644  ...  0.          0.
   0.        ]
 [ 1.4853516   1.7232581  -3.10292    ...  0.          0.
   0.        ]
 [ 0.8852539   1.0530717  -2.2728148  ...  0.          0.
   0.        ]]
Input shape:  (200000, 567)
Labels shape:  (200000,)
(200000, 2)
[0. 1.]


In [9]:
def datagenerator(inputs, labels, batchsize):
    while True:
        start = 0
        end = batchsize

        while start  < len(inputs): 
            # load your images from numpy arrays or read from directory
            x = inputs[start:end] 
            y = labels[start:end]
            yield x, y

            start += batchsize
            end += batchsize

In [10]:
from tensorflow import keras

outputFolder = path.join(datasetDir, 'Output_Keras_NodeFeatOnly')
Path(outputFolder).mkdir(parents=True, exist_ok=True)

tf.keras.backend.clear_session()
model = keras.Sequential(name="KerasModel_NodeFeatOnly")
inputDim = nFeatDim * maxNodeCount
model.add(keras.layers.InputLayer(input_shape=(inputDim,), name="input"))
model.add(keras.layers.Dense(16, activation='relu', name="dense1"))
model.add(keras.layers.Dense(16, activation='relu', name="dense2"))
model.add(keras.layers.Dense(2, activation='softmax', name="output"))
model.summary()

lossfunction = keras.losses.CategoricalCrossentropy()
optimizer = keras.optimizers.Adam(learning_rate=0.001)
modelcheckpoint = tf.keras.callbacks.ModelCheckpoint(filepath=path.join(outputFolder,'keras_nodeFeatOnly_bestmodel.h5'), monitor='val_loss', save_best_only=True, verbose=1)
csvlogger = tf.keras.callbacks.CSVLogger(filename=path.join(outputFolder, 'results_keras_nodeFeatOnly_bestmodel.csv'), separator=',', append=False)
callbacks = [modelcheckpoint, csvlogger]

model.summary()
model.compile(optimizer=optimizer, loss=lossfunction, metrics=['accuracy'])

Model: "KerasModel_NodeFeatOnly"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense1 (Dense)               (None, 16)                9088      
_________________________________________________________________
dense2 (Dense)               (None, 16)                272       
_________________________________________________________________
output (Dense)               (None, 2)                 34        
Total params: 9,394
Trainable params: 9,394
Non-trainable params: 0
_________________________________________________________________
Model: "KerasModel_NodeFeatOnly"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense1 (Dense)               (None, 16)                9088      
_________________________________________________________________
dense2 (Dense)               (None, 16)                272       
________

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_testAndVal, y_train, y_testAndVal = train_test_split(inputs, labels, test_size=0.3, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_testAndVal, y_testAndVal, test_size=0.33, shuffle=False)
print(f'train samples: {len(X_train)}')
print(f'validation samples: {len(X_val)}')
print(f'test samples: {len(X_test)}')

train samples: 140000
validation samples: 40200
test samples: 19800


In [12]:
batchsize=1024

import time
start = time.time()

history = model.fit(
    x = datagenerator(X_train, y_train, batchsize=batchsize),
    validation_data = datagenerator(X_val, y_val,batchsize=batchsize),
    steps_per_epoch = len(X_train)//batchsize,
    validation_steps = len(X_val)//batchsize,
    shuffle=False, # at creation from rootfiles -> graphs already shuffled dataset
    epochs = 30, # doesnt matter, since we use early stopping
    callbacks = callbacks
)

end = time.time() - start
print(f'training took {end:.2f} seconds ({end/60:.2f} minutes)')

Epoch 1/30
136/136 [==============================] - 3s 12ms/step - loss: 0.4030 - accuracy: 0.8168 - val_loss: 0.2719 - val_accuracy: 0.8945

Epoch 00001: val_loss improved from inf to 0.27195, saving model to /ceph/aissac/ntuple_for_graphs/prod_2018_v2_processed_v5_THESIS/trimmed_200000_and_cut_puppiWeightNoLep_greater_0_and_deltaR_smaller_0point5/Graphs_DYJetsToLL_M-50_genuineTaus_and_jets/Output_Keras_NodeFeatOnly/keras_nodeFeatOnly_bestmodel.h5
Epoch 2/30
136/136 [==============================] - 1s 9ms/step - loss: 0.2632 - accuracy: 0.8969 - val_loss: 0.2626 - val_accuracy: 0.8973

Epoch 00002: val_loss improved from 0.27195 to 0.26257, saving model to /ceph/aissac/ntuple_for_graphs/prod_2018_v2_processed_v5_THESIS/trimmed_200000_and_cut_puppiWeightNoLep_greater_0_and_deltaR_smaller_0point5/Graphs_DYJetsToLL_M-50_genuineTaus_and_jets/Output_Keras_NodeFeatOnly/keras_nodeFeatOnly_bestmodel.h5
Epoch 3/30
136/136 [==============================] - 1s 9ms/step - loss: 0.2574 - accu

Epoch 19/30
136/136 [==============================] - 1s 5ms/step - loss: 0.2327 - accuracy: 0.9077 - val_loss: 0.2396 - val_accuracy: 0.9054

Epoch 00019: val_loss improved from 0.24029 to 0.23958, saving model to /ceph/aissac/ntuple_for_graphs/prod_2018_v2_processed_v5_THESIS/trimmed_200000_and_cut_puppiWeightNoLep_greater_0_and_deltaR_smaller_0point5/Graphs_DYJetsToLL_M-50_genuineTaus_and_jets/Output_Keras_NodeFeatOnly/keras_nodeFeatOnly_bestmodel.h5
Epoch 20/30
136/136 [==============================] - 1s 8ms/step - loss: 0.2321 - accuracy: 0.9081 - val_loss: 0.2392 - val_accuracy: 0.9059

Epoch 00020: val_loss improved from 0.23958 to 0.23918, saving model to /ceph/aissac/ntuple_for_graphs/prod_2018_v2_processed_v5_THESIS/trimmed_200000_and_cut_puppiWeightNoLep_greater_0_and_deltaR_smaller_0point5/Graphs_DYJetsToLL_M-50_genuineTaus_and_jets/Output_Keras_NodeFeatOnly/keras_nodeFeatOnly_bestmodel.h5
Epoch 21/30
136/136 [==============================] - 1s 9ms/step - loss: 0.2318 

In [ ]:
#outputFolder = path.join(datasetDir, 'Output_Keras_NodeFeatOnly')

#model = keras.models.load_model(path.join(outputFolder, 'keras_nodeFeatOnly_bestmodel.h5'))

In [17]:
# NN output plot
predictions = model.predict(X_test)
#print(predictions)

matplotlib.rcParams.update({'font.size': 16})
lw = 2
xyLabelFontSize = 20
xLabelPad = 10
yLabelPad = 15

def createFigure():
    fig, ax = plt.subplots(figsize=(10,7))
    ax.tick_params(pad=7)
    return fig, ax

# TODO: check which order is actually signal (genuineTau) and which are background (fakeTau)
genuineTau_decisions = predictions[:,0]
fakeTau_decisions = predictions[:,1]

plt.figure(figsize=(9,7))

plt.hist(genuineTau_decisions, label='Genuine Taus', 
        histtype='step', # lineplot that's unfilled
        density=True, # normalize to form a probability density
        linewidth=lw)
plt.hist(fakeTau_decisions, label='Jets', 
        histtype='step', # lineplot that's unfilled
        density=True, linewidth=lw) # normalize to form a probability density
plt.xlabel('Neural Network output') # add x-axis label
plt.ylabel('Arbitrary units') # add y-axis label
plt.legend(loc="upper center") # add legend
plt.savefig(path.join(outputFolder, "NN_output.png"))
plt.clf()

from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(y_test.argmax(axis=1), predictions[:, 1])
roc_auc = auc(fpr, tpr) # area under curve (AUC), ROC = Receiver operating characteristic

fig, ax = createFigure()
ax.plot(fpr, tpr, label=f'ROC (area = {roc_auc:.2f})', linewidth=lw)
ax.plot([0, 1], [0, 1], '--', color='red', label='Luck', linewidth=lw)
ax.set_xlabel('False Positive Rate') 
ax.set_ylabel('True Positive Rate')
ax.legend()
ax.grid()
outputFilePath = path.join(outputFolder, 'ROC.png')
plt.savefig(outputFilePath)
plt.clf()

print("\n")
print(history.history)

# Plot accuracy of NN
fig, ax = createFigure()
ax.plot(history.history['accuracy'], label='Training', linewidth=lw)
ax.plot(history.history['val_accuracy'], label='Validation', linewidth=lw)
ax.set_ylabel('Accuracy', labelpad=xLabelPad, fontsize=xyLabelFontSize)
ax.set_xlabel('Epoch', labelpad=yLabelPad, fontsize=xyLabelFontSize)
ax.legend()
outputFilePath = path.join(outputFolder, 'accuracy.png')
plt.savefig(outputFilePath)
plt.clf()
# Plot loss of NN
fig, ax = createFigure()
ax.plot(history.history['loss'], label="Training",linewidth=lw)
ax.plot(history.history['val_loss'], label="Validation", linewidth=lw)
ax.set_ylabel('Loss', labelpad=xLabelPad, fontsize=xyLabelFontSize)
ax.set_xlabel('Epoch', labelpad=yLabelPad, fontsize=xyLabelFontSize)
ax.legend()
outputFilePath = path.join(outputFolder, 'epochloss.png')
plt.savefig(outputFilePath)
plt.clf()


# evaluate the model
_, train_acc = model.evaluate(X_train, y_train, verbose=1)
_, test_acc = model.evaluate(X_test, y_test, verbose=1)
_, valid_acc = model.evaluate(X_val, y_val, verbose=1)
print(f'Train: {train_acc:.3f}, Valid: {valid_acc:.3f}, Test: {test_acc:.3f}, AUC: {roc_auc:.3f}')



{'loss': [0.4029580056667328, 0.26321274042129517, 0.25740376114845276, 0.254263311624527, 0.2515096664428711, 0.2493506371974945, 0.24726329743862152, 0.2454703003168106, 0.24394653737545013, 0.24208103120326996, 0.24017147719860077, 0.23883184790611267, 0.237776979804039, 0.23663264513015747, 0.2354976385831833, 0.23467540740966797, 0.2340826392173767, 0.23337422311306, 0.23274736106395721, 0.23206248879432678, 0.2318488508462906, 0.2313382923603058, 0.23057131469249725, 0.2300124615430832, 0.22976182401180267, 0.22929516434669495, 0.22881312668323517, 0.22859112918376923, 0.22803953289985657, 0.22754377126693726], 'accuracy': [0.8167580962181091, 0.896917462348938, 0.8980111479759216, 0.8992199897766113, 0.900177001953125, 0.9010188579559326, 0.9020910263061523, 0.9025803208351135, 0.9032207131385803, 0.9040337800979614, 0.9050555229187012, 0.9053577780723572, 0.9058254957199097, 0.9061492681503296, 0.9066385626792908, 0.906998336315155, 0.9073293209075928, 0.9076027274131775, 0.9

<Figure size 648x504 with 0 Axes>

<Figure size 720x504 with 0 Axes>

<Figure size 720x504 with 0 Axes>

<Figure size 720x504 with 0 Axes>

In [ ]:
print(predictions)
print(predictions.argmax(1)[:100])
print(y_test.argmax(1)[:100])

In [ ]:
plt.figure(figsize=(9,7))

plt.hist(genuineTau_decisions, label='Genuine Taus', 
        histtype='step',
         density=True,
        linewidth=lw)
plt.hist(fakeTau_decisions, label='Jets', 
        histtype='step', 
         density=True,
         linewidth=lw)
plt.xlabel('Neural Network output') # add x-axis label
plt.ylabel('Arbitrary units') # add y-axis label
plt.legend(loc="upper center") # add legend
plt.show()

In [ ]:
print(genuineTau_decisions[:100])
print(fakeTau_decisions[:100])